In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df2022_full = pd.read_excel(r".\data\P1_2022_data.xlsx")

In [3]:
df2023_05 = pd.read_excel(r".\data\P1_2023_05_data.xlsx")

In [5]:
di_rename_col = {       '*************':                   'system_id',
                        '*****************************':   'comment_creation_date',
                        '*****************':               'project_name',
                        '*************':                   'set_id',
                        '*****************':               'set_rev',
                        '*********************':           'doc_id',
                        '*****************':               'doc_rev',
                        '**********************':          'doc_name',
                        '*************':                   'comment_type',
                        '************************':        'comment_detailed_type',
                        '***************':                 'comment_text',
                        '*************':                   'division',
                        '************':                    'response_text',
                        '************':                    'response_author',
                        '************************':        'response_date',
                        '****************':                'comment_status',
                        '********':                        'checklist_id',
                        '******************************':  'num_of_pages',
                        '***************':                 'comment_author'
                }

In [6]:
# 1st df
df2022_full = df2022_full.rename(columns=di_rename_col)

In [7]:
# 2nd df
df2023_05 = df2023_05.rename(columns=di_rename_col)

In [8]:
# missing data check 1st df
df2022_full.isna().mean()

system_id                0.000000
comment_creation_date    0.000000
project_name             0.000000
set_id                   0.009099
doc_id                   0.000000
set_rev                  0.000000
doc_name                 0.003817
comment_type             0.000000
comment_detailed_type    0.000000
comment_text             0.000000
division                 0.051434
response_text            0.133097
response_author          0.180329
response_date            0.139883
comment_status           0.132827
dtype: float64

In [9]:
# missing data check 2nd df
df2023_05.isna().mean()

system_id                0.000000
comment_creation_date    0.000000
project_name             0.000000
set_id                   0.000000
comment_type             0.000087
comment_detailed_type    0.001312
comment_text             0.000000
comment_author           0.096938
response_text            0.131234
response_author          0.172178
response_date            0.131409
set_rev                  0.000000
doc_id                   0.120472
doc_rev                  0.120472
doc_name                 0.123185
comment_status           0.131321
division                 0.036045
num_of_pages             0.000000
dtype: float64

# Concatenation of initial DataFrames

In [10]:
frames = [df2022_full, df2023_05]

In [11]:
df = pd.concat(frames, ignore_index=True)

# 1. 'system_id' replace:

In [14]:
def remap_system_id(df, col_name):
    i = 1
    for idx, row in df.iterrows():
        df.at[idx,col_name] = f'''ID-{'0'*(6 - len(str(i)))}{i}''' # id format: 'ID-00000N'
        i += 1

In [15]:
remap_system_id(df, 'system_id')

# 2. 'comment_creation_date' is not changed

# 3. 'project_name' replace:

In [16]:
df.project_name = 'P1'

# 4. 'set_id' replace:

In [17]:
def sets_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                num_zero = f'''{'0'*(6 - len({i}))}''' # format: '00000N'
                di[item] = f'''set-{int(int(str(item)[*:*])(formula))}-{num_zero}{i}''' # format: 'set-NNN-00000N'
                break
            except ValueError:
                di[item] = np.nan
                break       
        i += 1
    return di

In [18]:
set_ids = df['set_id'].unique()

In [19]:
df = df.replace({'set_id': sets_remap(set_ids)})

# 5. 'doc_id' replace:

In [20]:
def docs_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                num_zero = f'''{'0'*(6 - len({i}))}'''
                di[item] = f'''doc-{int(int(str(item)[*:*])(formula))}-{num_zero}{i}'''
                break
            except ValueError:
                di[item] = np.nan
                break
        i += 1
    return di

In [21]:
doc_ids = df['doc_id'].unique()

In [22]:
df = df.replace({'doc_id': docs_remap(doc_ids)})

# 6. 'set_rev' replace:

In [23]:
def rev_to_num(df, col_name):
    for idx, row in df.iterrows():
        while True:
            try:
                math.isnan(float(df.at[idx,col_name]))
                break
            except ValueError:
                df.at[idx,col_name] = str(df.at[idx,col_name])[1:]
                break    

In [24]:
rev_to_num(df, 'set_rev')

# 7. 'doc_rev' replace:

In [25]:
rev_to_num(df, 'doc_rev')

# 8. 'doc_name' replace:

In [26]:
def doc_name_remap(df, col_name):
    for idx, row in df.iterrows():
         while True:
            try:
                math.isnan(float(df.at[idx,col_name]))
                math.isnan(float(df.at[idx,'doc_id']))
                break
            except ValueError:
                df.at[idx,col_name] = f'''{df.at[idx,'doc_id']}_name'''
                break

In [27]:
doc_name_remap(df, 'doc_name')

In [28]:
df = df.replace({'doc_name': {'nan_name': np.nan}})

# 9. 'comment_type' replace:

In [29]:
types_name = df['comment_type'].unique()

In [30]:
def types_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                math.isnan(float(item))
                break
            except ValueError:
                di[item] = f'''type_{i}'''
                break
        i += 1
    return di

In [31]:
df = df.replace({'comment_type': types_remap(types_name)})

# 10. 'comment_detailed_type' replace:

In [32]:
detailed_types_name = df['comment_detailed_type'].unique()

In [33]:
def detailed_types_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                math.isnan(float(item))
                break
            except ValueError:
                di[item] = f'''subtype_{i}'''
                break    
        i += 1
    return di

In [34]:
df = df.replace({'comment_detailed_type': detailed_types_remap(detailed_types_name)})

# 11. 'comment_text' replace:

In [35]:
df.comment_text = 'comment_text'

# 12. 'comment_author' replace: replace:

In [36]:
comment_authors_name = df['comment_author'].unique()

In [37]:
def comment_authors_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                math.isnan(float(item))
                break
            except ValueError:
                di[item] = f'''comment_author_{i}'''
                break
        i += 1
    return di

In [38]:
df = df.replace({'comment_author': comment_authors_remap(comment_authors_name)})

# 13. 'division' replace:

In [39]:
divisions_name = df['division'].unique()

In [40]:
def divisions_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                math.isnan(float(item))
                break
            except ValueError:
                di[item] = f'''division_{i}'''
                break
        i += 1
    return di

In [41]:
df = df.replace({'division': divisions_remap(divisions_name)})

# 14. 'response_text' replace:

In [42]:
mask = df['response_text'].isnull()

In [43]:
df['response_text'] = df['response_text'].where(mask,'response_text')

# 15. 'response_author' replace:

In [44]:
response_authors_name = df['response_author'].unique()

In [45]:
def response_authors_remap(arr):
    di = {}
    i = 1
    for item in arr:
        while True:
            try:
                math.isnan(float(item))
                break
            except ValueError:
                di[item] = f'''response_author_{i}'''
                break
        i += 1
    return di

In [46]:
df = df.replace({'response_author': response_authors_remap(response_authors_name)})

# 16. 'response_date' is not changed

# 17. 'response_author' replace:

In [50]:
comment_status_name = df['comment_status'].unique()

In [52]:
di_comment_status_name = {'*****':'accepted','********':'fixed','********':'not_accepted',
                          '*******':'not_accepted','*******':fixed','******':'accepted'}

In [53]:
df = df.replace({'comment_status': di_comment_status_name})

# 18. 'num_of_pages' is not changed

# Result check

In [54]:
df

,system_id,comment_creation_date,project_name,set_id,doc_id,set_rev,doc_name,comment_type,comment_detailed_type,comment_text,division,response_text,response_author,response_date,comment_status,comment_author,doc_rev,num_of_pages
0,ID-000001,2022-05-31 11:25:02,P1,set-65-000001,doc-65-000001,01,doc-65-000001_name,type_1,subtype_1,comment_text,division_1,response_text,response_author_1,2022-06-08 08:08:33,accepted,NaN,NaN,NaN
1,ID-000002,2022-05-31 07:43:29,P1,set-60-000002,doc-60-000002,02,doc-60-000002_name,type_1,subtype_2,comment_text,division_2,response_text,response_author_2,2022-06-14 12:07:00,fixed,NaN,NaN,NaN
2,ID-000003,2022-05-31 07:23:09,P1,set-60-000003,doc-60-000003,01,doc-60-000003_name,type_1,subtype_3,comment_text,division_3,response_text,response_author_3,2022-06-30 12:40:26,fixed,NaN,NaN,NaN
3,ID-000004,2022-05-31 07:24:13,P1,set-60-000003,doc-60-000003,01,doc-60-000003_name,type_1,subtype_3,comment_text,division_3,response_text,response_author_3,2022-06-30 12:42:23,fixed,NaN,NaN,NaN
4,ID-000005,2022-06-01 11:08:10,P1,set-60-000004,doc-60-000004,02,doc-60-000004_name,type_2,subtype_4,comment_text,division_4,response_text,response_author_4,2022-06-06 12:33:28,accepted,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37361,ID-037362,2023-05-30 14:08:40,P1,set-60-000003538,doc-60-000007988,01,doc-60-000007988_name,type_1,subtype_24,comment_text,NaN,response_text,response_author_367,2023-05-31 13:02:05,not_accepted,comment_author_19,01,26.0
37362,ID-037363,2023-05-30 14:16:52,P1,set-55-00000699,doc-55-000001027,04,doc-55-000001027_name,type_3,subtype_17,comment_text,division_67,response_text,response_author_181,2023-05-31 08:43:17,fixed,comment_author_21,04,27.0
37363,ID-037364,2023-05-30 14:24:39,P1,set-65-000002177,doc-65-000003828,04,doc-65-000003828_name,type_5,subtype_12,comment_text,division_11,response_text,response_author_30,2023-06-01 10:35:32,fixed,comment_author_17,04,282.0
37364,ID-037365,2023-05-30 14:57:53,P1,set-65-000004306,doc-65-000007089,05,doc-65-000007089_name,type_5,subtype_12,comment_text,division_18,response_text,response_author_32,2023-06-02 07:35:22,accepted,comment_author_17,05,29.0


In [55]:
df.isna().mean()

system_id                0.000000
comment_creation_date    0.000000
project_name             0.000000
set_id                   0.019831
doc_id                   0.055505
set_rev                  0.000000
doc_name                 0.055505
comment_type             0.000027
comment_detailed_type    0.000401
comment_text             0.000000
division                 0.046727
response_text            0.132527
response_author          0.177835
response_date            0.137291
comment_status           0.132366
comment_author           0.723760
doc_rev                  0.730959
num_of_pages             0.694107
dtype: float64

# Change comments quantity (extarct **% random sample):

In [56]:
df_sample = df.sample(int(len(df.index)*(X))).reset_index(drop=True)

In [57]:
df_sample.isna().mean()

system_id                0.000000
comment_creation_date    0.000000
project_name             0.000000
set_id                   0.020446
doc_id                   0.056950
set_rev                  0.000000
doc_name                 0.056950
comment_type             0.000000
comment_detailed_type    0.000214
comment_text             0.000000
division                 0.046513
response_text            0.134507
response_author          0.178933
response_date            0.139378
comment_status           0.134293
comment_author           0.723010
doc_rev                  0.731360
num_of_pages             0.693839
dtype: float64

# Write result file in 'csv'

In [58]:
df_sample.to_csv(r'.\data\result_data.csv', sep=',')